# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [22]:
# Load dataset
file_path = "dataset_Financial_Transactions_klasifikasi.csv"
df = pd.read_csv(file_path)

In [23]:
# Melihat distribusi pada Cluster
cluster_distribution = df["Cluster"].value_counts()

# Encoding TransactionType menggunakan One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=["TransactionType"])

# Menampilkan distribusi Cluster dan hasil encoding awal
cluster_distribution, df_encoded.head()


(Cluster
 3    9442
 1    9406
 2    9309
 0    9260
 Name: count, dtype: int64,
    TransactionID  AccountID            Timestamp  TransactionAmount  \
 0          16633       5817  2016-01-01 03:47:23            2446.41   
 1          23660       9291  2016-01-01 04:20:25            2640.83   
 2          11806       9618  2016-01-01 05:12:44             574.82   
 3          27498       2288  2016-01-01 05:48:42            1740.12   
 4           9345       2688  2016-01-01 06:26:04             292.43   
 
    AccountBalance  Hour  DayOfWeek  Month  Cluster  TransactionType_deposit  \
 0        96273.47   3.0          4    1.0        1                    False   
 1        98629.95   4.0          4    1.0        1                    False   
 2        65602.63   5.0          4    1.0        3                    False   
 3        81461.66   5.0          4    1.0        3                    False   
 4        18084.81   6.0          4    1.0        3                    False   
 
   

In [24]:
# Hapus kolom yang tidak dibutuhkan
df_cleaned = df.drop(columns=['TransactionID', 'AccountID', 'Timestamp'])

In [25]:
# One-Hot Encoding untuk TransactionType
df_encoded = pd.get_dummies(df_cleaned, columns=['TransactionType'], drop_first=True)

# Normalisasi fitur numerik
scaler = MinMaxScaler()
numerical_features = ['TransactionAmount', 'AccountBalance', 'Hour', 'DayOfWeek', 'Month']
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

In [26]:
# Periksa hasil preprocessing
df_encoded.head()

,TransactionAmount,AccountBalance,Hour,DayOfWeek,Month,Cluster,TransactionType_payment,TransactionType_transfer,TransactionType_withdrawal
0,0.489214,0.962761,0.130435,0.666667,0.0,1,False,True,False
1,0.528109,0.986327,0.173913,0.666667,0.0,1,False,True,False
2,0.114790,0.656042,0.217391,0.666667,0.0,3,False,False,True
3,0.347916,0.814638,0.217391,0.666667,0.0,3,True,False,False
4,0.058296,0.180846,0.260870,0.666667,0.0,3,False,True,False


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [27]:
# Memisahkan fitur dan target
X = df_encoded.drop(columns=["Cluster"])
y = df_encoded["Cluster"]

# Membagi data menjadi 80% train dan 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [28]:
# Menampilkan bentuk akhir dataset setelah preprocessing
X.shape, y.shape

((37417, 8), (37417,))

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [29]:
# Melatih model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [30]:
svm_model = SVC().fit(X_train, y_train)

Saya memilih algotitma RandomForest dan KNN sebagai perbandingan algoritma dikarenakan keduanya memiliki karakteristik pelatihan untuk data yang berbeda. sehingga saya dapat mengetahui karakteristik data yang saya miliki lebih cocok menggunakan algorima apa.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [31]:
# Memprediksi pada data uji
y_pred = rf_model.predict(X_test)

In [32]:
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)


accuracy, f1, report

(0.9718065205772314,
 0.9718102478935182,
 '              precision    recall  f1-score   support\n\n           0       0.96      0.97      0.97      1852\n           1       0.97      0.97      0.97      1881\n           2       0.97      0.98      0.97      1862\n           3       0.98      0.97      0.98      1889\n\n    accuracy                           0.97      7484\n   macro avg       0.97      0.97      0.97      7484\nweighted avg       0.97      0.97      0.97      7484\n')

In [33]:
y_pred = svm_model.predict(X_test)

In [34]:
# Evaluasi model
accuracy_svm = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
report_svm = classification_report(y_test, y_pred)
conf_matrix_svm = confusion_matrix(y_test, y_pred)

accuracy_svm, report_svm, conf_matrix_svm, f1

(0.9887760555852485,
 '              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99      1852\n           1       0.99      0.98      0.99      1881\n           2       0.99      0.99      0.99      1862\n           3       0.99      0.99      0.99      1889\n\n    accuracy                           0.99      7484\n   macro avg       0.99      0.99      0.99      7484\nweighted avg       0.99      0.99      0.99      7484\n',
 array([[1828,    6,    9,    9],
        [  13, 1849,    8,   11],
        [   6,    6, 1845,    5],
        [   3,    6,    2, 1878]]),
 0.9887720882787878)

Hasil yang saya dapat cukup memuaskan dikarenakan saya dapat secara eksplisit mengetahui jenis algoritma seperti apa yang cocok untuk dataset yang saya miliki.

Perbandingan hasil evaluasi

1. Random Forest :
- accuracy = 0.9718065205772314
- F1 score = 97%

2. Support Vector Machine : 
- accuracy = 0.9887760555852485
- F1 score = 99%


## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [35]:
# Menentukan parameter grid untuk tuning
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

# Inisialisasi model Random Forest
rf = RandomForestClassifier(random_state=42)

# Grid Search dengan 3-fold cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Menampilkan hyperparameter terbaik
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

Fitting 3 folds for each of 81 candidates, totalling 243 fits


({'max_depth': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 200},
 np.float64(0.9683961425570252))

In [36]:
# Hyperparameter tuning untuk SVM
param_grid_svm = {
    "C": [0.1, 1, 10, 100],  # Regularisasi
    "kernel": ["linear", "rbf", "poly", "sigmoid"],  # Jenis kernel
    "gamma": ["scale", "auto"],  # Parameter gamma untuk kernel non-linear
}

# Grid Search untuk mencari parameter terbaik
grid_search_svm = GridSearchCV(SVC(), param_grid_svm, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
grid_search_svm.fit(X_train, y_train)

# Menampilkan hasil tuning
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_

best_params_svm, best_score_svm


Fitting 3 folds for each of 32 candidates, totalling 96 fits


({'C': 100, 'gamma': 'scale', 'kernel': 'linear'},
 np.float64(0.9955567378030205))

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [39]:
# Melatih model Random Forest
rf_model = RandomForestClassifier(n_estimators=200, 
max_depth=20, min_samples_leaf=1, min_samples_split=2, random_state=42)

rf_model.fit(X_train, y_train)

y_pred_tuning = rf_model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred_tuning)
report = classification_report(y_test, y_pred_tuning)


accuracy, report

(0.9720737573490112,
 '              precision    recall  f1-score   support\n\n           0       0.97      0.97      0.97      1852\n           1       0.98      0.97      0.97      1881\n           2       0.97      0.98      0.97      1862\n           3       0.98      0.98      0.98      1889\n\n    accuracy                           0.97      7484\n   macro avg       0.97      0.97      0.97      7484\nweighted avg       0.97      0.97      0.97      7484\n')

In [38]:
svm_model = SVC( C= 100, kernel= "linear",  gamma= "scale")
svm_model.fit(X_train, y_train)

y_pred_tuning = svm_model.predict(X_test)

# Evaluasi model
accuracy_svm = accuracy_score(y_test, y_pred_tuning)
report_svm = classification_report(y_test, y_pred_tuning)
conf_matrix_svm = confusion_matrix(y_test, y_pred_tuning)

accuracy_svm, report_svm, conf_matrix_svm

(0.9970603955104222,
 '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      1852\n           1       1.00      1.00      1.00      1881\n           2       1.00      1.00      1.00      1862\n           3       1.00      1.00      1.00      1889\n\n    accuracy                           1.00      7484\n   macro avg       1.00      1.00      1.00      7484\nweighted avg       1.00      1.00      1.00      7484\n',
 array([[1844,    3,    3,    2],
        [   3, 1875,    2,    1],
        [   1,    0, 1859,    2],
        [   3,    1,    1, 1884]]))

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.


1. Perbandingan Sebelum & Setelah Tuning
- pada kedua algoritma setelah tuning dapat terlihat peningkatan dari akurasi mode dan f1 score, namun peningkatan tidak terlalu signifikam
- Rf : 
   - Accurasi: 97,1% -> 97,2%
   - F1 score: 96 - 97% -> 97 - 98%

- SVM 
   - Accurasi: 98% -> 99%
   - F1 score: 99% -> 100%

2. Identifikasi Kelemahan Model
- Model Rf
   - Precision dan Recall tinggi di semua kelas, menunjukkan model cukup seimbang.
   - Overfitting tidak terlalu terlihat, karena akurasi validasi hampir sama dengan data uji.

- Model Svm
   - Berbeda dengan Random Forest  SVM tidak memiliki fitur importance, sehingga sulit mengetahui faktor utama yang mempengaruhi klasifikas
   - Kurang efisien untuk jumlah data yang besar


3. Rekomendasi Tindakan Lanjutan
 - 1. Coba Model Lain
      - Model dengan type yang mirip Random forest mungkin dapat meningkatkan hasil.
 - 2. Lakukan Feature engineering
    - Mungkin ada fitur yang kurang relevan atau bisa ditransformasi lebih baik.
 - 3. Hyperparameter Tuning dengan Bayesian Optimization
    - Bayesian Optimization (Optuna) bisa lebih optimal dalam mencari hyperparameter terbaik.